## Exercise 05 : Pandas optimizations

In [37]:
%autosave 60
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Autosaving every 60 seconds


### Read the fines.csv

In [38]:
df = pd.read_csv('../data/fines.csv')
df

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.0,Ford,Focus,1989
1,E432XX77RUS,1,6500.0,Toyota,Camry,1995
2,7184TT36RUS,1,2100.0,Ford,Focus,1984
3,X582HE161RUS,2,2000.0,Ford,Focus,2015
4,92918M178RUS,1,5700.0,Ford,Focus,2014
...,...,...,...,...,...,...
925,SCHOOL21000RUS,1,5800.0,Toyota,Focus,2017
926,SCHOOL21001RUS,1,41700.0,Ford,Focus,2008
927,SCHOOL21002RUS,1,200.0,Ford,Focus,1985
928,SCHOOL21003RUS,2,800.0,Ford,Focus,2000


### Iterations

using loop:

In [39]:
%%time
new_col = []

for i in range(0, len(df)):
    new_col.append(df.iloc[i]['Fines'] / (df.iloc[i]['Refund'] * df.iloc[i]['Year']))

df['Fines / Refund * Year'] = new_col

CPU times: user 239 ms, sys: 4.06 ms, total: 243 ms
Wall time: 245 ms


using iterrows:

In [40]:
%%time
new_col = []

for index, row in df.iterrows():
    new_col.append(row['Fines'] / (row['Refund'] * row['Year']))

df['Fines / Refund * Year'] = new_col

CPU times: user 50.7 ms, sys: 1.64 ms, total: 52.4 ms
Wall time: 51.6 ms


using apply:

In [41]:
%%time
df['Fines / Refund * Year'] = df.apply(
    lambda row: row['Fines'] / (row['Refund'] * row['Year']), axis=1
    )

CPU times: user 13.5 ms, sys: 1.21 ms, total: 14.7 ms
Wall time: 14.2 ms


using Series objects from the dataframe:

In [42]:
%%time
df['Fines / Refund * Year'] = df['Fines'] / (df['Refund'] * df['Year'])

CPU times: user 786 µs, sys: 59 µs, total: 845 µs
Wall time: 888 µs


using values:

In [43]:
%%time
df['Fines / Refund * Year'] = df['Fines'].values / (
    df['Refund'].values * df['Year'].values
    )

CPU times: user 294 µs, sys: 6 µs, total: 300 µs
Wall time: 305 µs


### Indexing

get a row for a specific CarNumber

In [44]:
%%time
df.loc[df['CarNumber'] == 'M0299X197RUS']

CPU times: user 1.07 ms, sys: 126 µs, total: 1.2 ms
Wall time: 1.11 ms


,CarNumber,Refund,Fines,Make,Model,Year,Fines / Refund * Year
190,M0299X197RUS,2,67200.0,Ford,Focus,1995,16.842105
445,M0299X197RUS,2,19200.0,Ford,Focus,2005,4.788030
473,M0299X197RUS,1,9600.0,Ford,Focus,2007,4.783259
725,M0299X197RUS,1,8300.0,Ford,Focus,1989,4.172951


set the index in your dataframe with CarNumber

In [45]:
df1 = df.set_index('CarNumber')

again, get a row for the same CarNumber

In [46]:
%%time
df1.loc['M0299X197RUS']

CPU times: user 1.7 ms, sys: 350 µs, total: 2.05 ms
Wall time: 2.27 ms


,Refund,Fines,Make,Model,Year,Fines / Refund * Year
CarNumber,,,,,,
M0299X197RUS,2,67200.0,Ford,Focus,1995,16.842105
M0299X197RUS,2,19200.0,Ford,Focus,2005,4.788030
M0299X197RUS,1,9600.0,Ford,Focus,2007,4.783259
M0299X197RUS,1,8300.0,Ford,Focus,1989,4.172951


### Downcasting

run df.info(memory_usage=’deep’), pay attention to the Dtype and the mem- ory usage

In [47]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CarNumber              930 non-null    object 
 1   Refund                 930 non-null    int64  
 2   Fines                  930 non-null    float64
 3   Make                   930 non-null    object 
 4   Model                  919 non-null    object 
 5   Year                   930 non-null    int64  
 6   Fines / Refund * Year  930 non-null    float64
dtypes: float64(2), int64(2), object(3)
memory usage: 203.8 KB


make a copy() of your initial dataframe into another dataframe optimized

downcast from float64 to float32 for all the columns

downcast from int64 to the smallest numerical dtype possible

In [48]:
df_optimized = df.copy()
df_optimized['Fines'] = pd.to_numeric(df_optimized['Fines'], downcast='float')
df_optimized['Refund'] = pd.to_numeric(df_optimized['Refund'], downcast='integer')
df_optimized['Year'] = pd.to_numeric(df_optimized['Year'], downcast='integer')

run info(memory_usage=’deep’) for your new dataframe, pay attention to the Dtype and the memory usage

In [49]:
df_optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CarNumber              930 non-null    object 
 1   Refund                 930 non-null    int8   
 2   Fines                  930 non-null    float32
 3   Make                   930 non-null    object 
 4   Model                  919 non-null    object 
 5   Year                   930 non-null    int16  
 6   Fines / Refund * Year  930 non-null    float64
dtypes: float32(1), float64(1), int16(1), int8(1), object(3)
memory usage: 188.4 KB


### Categories

change the object type columns to the type category

In [50]:
df_optimized['Make'] = df_optimized['Make'].astype('category')
df_optimized['Model'] = df_optimized['Model'].astype('category')

check the memory usage, it probably has a decrease of 2-3 times compared to the initial dataframe

In [51]:
df_optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   CarNumber              930 non-null    object  
 1   Refund                 930 non-null    int8    
 2   Fines                  930 non-null    float32 
 3   Make                   930 non-null    category
 4   Model                  919 non-null    category
 5   Year                   930 non-null    int16   
 6   Fines / Refund * Year  930 non-null    float64 
dtypes: category(2), float32(1), float64(1), int16(1), int8(1), object(1)
memory usage: 79.7 KB


### Memory clean

using %reset_selective and the library gc clean the memory of your initial dataframe only

In [52]:
%reset_selective -f df
df

NameError: name 'df' is not defined